In [1]:
from ntk import empirical_ntk_ntk_vps, empirical_ntk_jacobian_contraction
from NeuralNet import MTRAgent, ReinforceAgent
from MtrNet import MtrNet
import torch
from original import OriginalMTR
from torch.func import functional_call, vmap, vjp, jvp, jacrev
GAME_NAME = "Toy"
"""
            INITIALIZING AGENTS and ENVIRONMENT
"""
"""-------------------------------------------------------------------"""
from enviroment import *
env, obs_dim, action_dim = game_setup(GAME_NAME )

In [2]:
"""
                 SETUP AGENT and TRAINING
"""
"""REINFORCE"""
BETA_REIN_FULL = 0.1 #Betas are scalar by which we multiply bias terms in NN at initialization. They are used to control Chaos order.
BETA_REIN_RESNET = BETA_REIN_FULL
LEARNING_RATE_REIN = 0.0001
HORIZON_REIN = 60
HIDDEN_DIM_REIN = 32
"""MATRYOSHKA"""
BETA_MTR_FULL = 0.1
LEARNING_RATE_MTR = 0.0001
HORIZON_MTR= HORIZON_REIN
HIDDEN_DIM_MTR = 32
TAU = 1
"""ResNet"""
BETA_MTR_RESNET = BETA_MTR_FULL
HIDDEN_DIM_RESNET = 16
LEARNING_RATE_RESNET = LEARNING_RATE_MTR * 10
"""Original"""
HIDDEN_DIM_ORIGINAL = 16
HORIZON_ORIGINAL = HORIZON_REIN
BETA_ORIGINAL = 0.1
TAU_ORIGINAL = 0.4
LEARNING_RATE_ORIGINAL = LEARNING_RATE_RESNET
"""Training"""
NUM_EPOCHES = 500
N_EPISODES = 10 # number of episodes per epoch. Used for both: Reinforce and Matryoshka
"""
                    INITIALIZING AGENTS
"""
"""-------------------------------------------------------------------"""
from NeuralNet import ReinforceAgent, MTRAgent
from MtrNet import ReinforceMtrNetAgent, MtrNetAgent, ShortLongAgent
import torch

#Reinforce Full Connected
agent1 = ReinforceAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_REIN,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_REIN, game_name=GAME_NAME, beta = BETA_MTR_FULL)
agent1.policy.ntk_init(beta=agent1.beta)
#Reinforce MTRNet (Custom ResNet)
agent2 = ReinforceMtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_REIN, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_REIN_RESNET)
agent2.policy.ntk_init(beta=agent2.beta)
#MTR extra-dimension
agent3 = MTRAgent(obs_dim, action_dim, hidden_dim= HIDDEN_DIM_MTR , horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_MTR, game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_FULL)
agent3.policy.ntk_init(beta=agent3.beta)
#MTR with MTRNet
agent4 = MtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET)
agent4.policy.ntk_init(beta=agent4.beta)
from original import OriginalMtrAgent
# ORIGINAL
agent5 = OriginalMtrAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_ORIGINAL,horizon= HORIZON_ORIGINAL, learning_rate= LEARNING_RATE_ORIGINAL , game_name= GAME_NAME, tau=TAU_ORIGINAL, beta=BETA_ORIGINAL)
agent5.policy.ntk_init(beta=agent5.beta)
# MTRNet with dynamical number of parameters
agent6 = MtrNetAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_MTR,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET, dynamical_layer_param=True)
agent6.policy.ntk_init(beta=agent6.beta)
#ShortLongNet
agent7 = ShortLongAgent(obs_dim,action_dim, hidden_dim=HIDDEN_DIM_RESNET,horizon= HORIZON_MTR, learning_rate= LEARNING_RATE_RESNET , game_name= GAME_NAME, tau=TAU, beta=BETA_MTR_RESNET)
agent7.policy.ntk_init(beta=agent7.beta)
Agents = [agent2, agent4, agent5, agent6]

In [3]:
from utils import run_episodes_mtr
episodes = run_episodes_mtr(agent3, env,n_episodes=1)
states, _, _, _, _ = zip(*episodes[0])
x1_3 = states[0]
x2_3 = states[1]
x1 = x1_3[:-1]
x2 = x2_3[:-1]

In [4]:

print("States")
print("x1:",x1)
print("x2:", x2)
print("")
print(agent3.name)
print(agent3.ntk(x1_3,x2_3)[0])

print(agent1.name)
print(agent1.ntk(x1,x2)[0])
step = 1
for agent in Agents:
    print(agent.name)
    print(agent.ntk(x1,x2, step)[0])

States
x1: [1. 0. 0. 0. 0. 0. 0. 0. 0.]
x2: [0. 0. 1. 0. 0. 0. 0. 0. 0.]

MTR
tensor([[[ 4.0162, -0.1050],
         [-0.1050,  3.7462]]])
REIN
tensor([[[2.4769, 0.0103],
         [0.0103, 2.4800]]])
ReinMtrNet
tensor([[[ 2.4379, -0.0388],
         [-0.0388,  2.4591]]])
MtrNet
tensor([[[ 1.3998, -0.0497],
         [-0.0497,  1.4191]]])
OriginalMtr
tensor([[[ 2.0578, -0.0929],
         [-0.0929,  2.2116]]])
MtrNet
tensor([[[ 1.5493, -0.1329],
         [-0.1329,  2.0110]]])


In [10]:
step = 1
print(agent6.policy.count_parameters_in_block(agent6.horizon- step))
agent6.ntk(x2,x1,step,softmax=False, show_dim_jac=True)[0]

86
Layer blocks.5.Q.0.weight has jacobian shape:  torch.Size([1, 2, 63])
Layer blocks.5.Q.0.bias has jacobian shape:  torch.Size([1, 2, 7])
Layer blocks.5.Q.2.weight has jacobian shape:  torch.Size([1, 2, 14])
Layer blocks.5.Q.2.bias has jacobian shape:  torch.Size([1, 2, 2])


tensor([[[ 1.5493, -0.1329],
         [-0.1329,  2.0110]]])

In [29]:
print(agent4.policy.count_parameters_in_block(agent4.horizon- step))
agent4.ntk(x2,x1,step,softmax=True, show_dim_jac=True)[0]

194
Layer blocks.5.Q.0.weight has jacobian shape:  torch.Size([1, 2, 144])
Layer blocks.5.Q.0.bias has jacobian shape:  torch.Size([1, 2, 16])
Layer blocks.5.Q.2.weight has jacobian shape:  torch.Size([1, 2, 32])
Layer blocks.5.Q.2.bias has jacobian shape:  torch.Size([1, 2, 2])


tensor([[[ 0.1724, -0.1724],
         [-0.1724,  0.1724]]], grad_fn=<SelectBackward0>)

In [9]:
step = 56
agent7.block_idx(agent7.horizon-step)

[ 1.8  3.6  6.   9.6 15.6 22.8 30.  39.  48. ]

[0.03, 0.06, 0.1, 0.16, 0.26, 0.38, 0.5, 0.65, 0.8]


2

In [5]:
[0.1,0.2, 1] * 3

[0.1, 0.2, 1, 0.1, 0.2, 1, 0.1, 0.2, 1]

In [ ]:
"""
        TRAINING (REINFORCE) PARALLEL
        Remark: NOT WORKING, in process...
"""

LEARNING_RATE = 0.01
NUM_EPOCHES = 30
N_EPISODES = 20 # ^number of episodes per epoch
"""-------------------------------------------------------------------"""
import torch.optim as optim
from try_grad_policy import train_policy
import multiprocessing

optimizer = optim.Adam(policy.parameters(), lr=LEARNING_RATE)  # Define the optimizer
optimizer_ntk = optim.Adam(ntk_policy.parameters(), lr=LEARNING_RATE)  # Define the optimizer


initialize_env(env)

output_queue1 = multiprocessing.Queue()
output_queue2 = multiprocessing.Queue()
run_process1 = multiprocessing.Process(target=run_episodes, args=(policy, env,output_queue1, N_EPISODES))
run_process2 = multiprocessing.Process(target=run_episodes, args=(ntk_policy, env,output_queue2, N_EPISODES))


# Train for a number of epochs
for epoch in range(NUM_EPOCHES ):
    run_process1.start()  # Collect episodes
    run_process2.start()
    # Join the processes to wait for them to finish
    run_process1.join()
    run_process2.join()
    # Get the outputs from both processes
    episodes = output_queue1.get()
    ntk_episodes = output_queue2.get()
    loss = train_policy(policy, optimizer, episodes)  # Update the policy based on the episodes
    ntk_loss = train_policy(ntk_policy, optimizer_ntk, ntk_episodes)  # Update the ntk_policy based on the episodes
    print(f'Episode {epoch + 1}, Total Loss: {BETA1}={loss} and {BETA2}={ntk_loss} ')
close_env(env)
"-------------------TEST-------------------"
env = gym.make('CartPole-v1', render_mode = "human" )
run_episodes(policy, env, n_episodes=10)